In [ ]:
import xml.etree.ElementTree
import os, os.path, shutil
os.sep = '/'
from gedcom import Gedcom, GedcomReader, GedcomWriter

def copy_tree_image(img_path):
    shutil.copy(src=img_path, dst='static/tree/tree_img'+os.path.splitext(img_path)[-1])
    shutil.copy(src=os.path.splitext(img_path)[0]+'.xml', dst='../data/tree_img.xml')



import random, string
def random_string(length, letters=None):
    letters = string.ascii_lowercase + string.digits
    return ''.join(random.choice(letters) for i in range(length))
            
def pack_documents(gedcom, folder):
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder)
    relative_pathes = {}
    for person in gedcom.persons:
        person_uid = person.get('_UID')
        person_folder = os.path.join(folder, person_uid)
        all_documents = [doc for doc in person.documents if doc.get('DFLT') == 'T'] #+ [doc for event in person.events for doc in event.documents]
        for document in all_documents:
            document_path = document.get('FILE')
            if document_path is None or not os.path.exists(document_path):
                document['FILE'] = ""
                continue
            if document_path not in relative_pathes:
                ext = os.path.splitext(document_path)[-1]
                new_document_name = random_string(10)+ext
                if not os.path.exists(person_folder):
                    os.makedirs(person_folder)
                new_document_path = os.path.join(person_folder, new_document_name)   
                shutil.copy(src=document_path, dst=new_document_path)
                rel_document_path = os.path.join(person_uid, new_document_name)
                relative_pathes[document_path] = rel_document_path
            
            document['FILE'] = relative_pathes[document_path]


In [ ]:
# читаем gedcom
gedcom = GedcomReader().read_gedcom(r'D:/Документы/My Family/site/tree.ged')
treexml = xml.etree.ElementTree.parse('D:/Документы/My Family/site/tree.xml').getroot()

copy_tree_image(r'D:/Документы/My Family/site/tree_img.png')
set_default_documents_for_persons(gedcom, treexml)
set_documents_to_marriage(gedcom)
set_divorce_events_to_families(gedcom, treexml)
pack_documents(gedcom, 'static/tree/tr_tree.files')

GedcomWriter().write_gedcom(gedcom, '../data/tr_tree.ged')

In [ ]:
import pandas
events = pandas.read_csv('data/events.csv', sep=';', skiprows=1)
persons = pandas.read_csv('data/persons.csv', sep=';', skiprows=1)
documents = pandas.read_csv('data/documents.csv', sep=';', skiprows=1)

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
print(len(gedcom.persons))
for family in gedcom.families:
    print(family)

In [ ]:
from app_utils import first_or_default

gedcom = GedcomReader().read_gedcom('data/exmp.ged')
person = gedcom.persons[0]
print([(e.event_type, e) for e in person.events])
death_event = first_or_default(person.events, lambda e: e.event_type == "DEAT")
print(death_event)
if death_event is not None:
    print('death')

In [ ]:
import upload
archive = upload.create_package(r'D:\site', 'data')
print(archive)

In [ ]:
def split_binary_file(filename, chunk_size=50*1024*1024):
    """split file into chunks of size"""
    
    f = open(filename, 'rb')
    data = f.read()
    f.close()

    count_chunks = int(len(data)/chunk_size + bool(len(data)%chunk_size))
    for i in range(0, count_chunks):
        chunk_filename = filename+".{0:000}".format(i+1)
        f = open(chunk_filename, 'wb+')
        f.write(data[i*chunk_size: (i+1)*chunk_size])
        f.close()

In [ ]:
split_binary_file(r'D:\site\data.zip')

In [ ]:
import upload
archive = r'D:\site\data.zip'
upload.load_package(archive, 
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\src\static\tree',
                    r'C:\Users\Ksenia\Documents\IPython Notebooks\genotree\data\tree')

In [ ]:
from gedcom import Gedcom, GedcomReader, GedcomWriter
gedcom = GedcomReader().read_gedcom('D:/site/tree.ged')

In [ ]:
gedcom = GedcomReader().read_gedcom('data/exmp.ged')